# **Assignment 4**

## **ECON8502: Structural Microeconometrics**

### *Conor Bayliss*

#### **Setup**

*This week you will estimate a simple hidden Markov model using Expectation-Maximisation.*

*Here is the model. There is a discrete state variable $k\in\{1,2,3,...,K\}$ and a binary outcome $j\in\{0,1\}$ that:*
1. *Is determined probabilistically by the state; and*
2. *Moves the state up one grid point if $j=1$, i.e. $k_{t+1} = \min\{K,k_{t+j}\}$*

*Let $p$ be a K-dimensional vector where $p_k$ holds the probability that $j=1$ given that the model is in state $k$.*

The state $k$ is never observed and the outcome $j$ is only observed half the time (i.e. it is missing with probability 0.5). Thus, define $j^{*}$ to be:
$$
j^{*} = 
\begin{cases} 
j & \text{with probability } 0.5 \\
-1 & \text{with probability } 0.5
\end{cases}
$$

*Our task is to estimate the vector of outcome probabilities $p$ non-parametrically using the EM algorithm with the Forward-Back routine to calculate the E-step*.

*The code below calcualtes $p$ and simulates panel data*
$$
(j^*_{nt})_{t=1,n=1}^{T,N}
$$
*To start with, assume $k_{n,1} = 1, \forall n$.*

In [1]:
using Random, Distributions
K = 5
Pj = 1 ./ (1 .+ exp.(LinRange(-1,1,K))) #<- choice probability as a function of k
@show Pj
knext(k,j,K) = min(K,k+j) 

function simulate(N,T,Pj)
    J = zeros(T,N)
    K = length(Pj)
    for n in axes(J,2)
        k = 1
        for t in axes(J,1)
            j = rand()<Pj[k]
            # record j probabilistically
            if rand()<0.5
                J[t,n] = -1
            else
                J[t,n] = j
            end
            # update state:
            k = knext(k,j,K)
        end
    end
    return J
end

J_data = simulate(1000,10,Pj)

Pj = [0.7310585786300049, 0.6224593312018546, 0.5, 0.3775406687981454, 0.2689414213699951]


10×1000 Matrix{Float64}:
 -1.0  -1.0  -1.0   1.0  -1.0  -1.0  …   1.0  -1.0   1.0  -1.0  -1.0  -1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0     -1.0   1.0   1.0  -1.0   1.0  -1.0
 -1.0  -1.0  -1.0   1.0  -1.0  -1.0      0.0  -1.0  -1.0  -1.0  -1.0  -1.0
  1.0   1.0   0.0  -1.0  -1.0   1.0     -1.0  -1.0   1.0  -1.0   1.0  -1.0
 -1.0   0.0   0.0  -1.0  -1.0  -1.0      1.0  -1.0  -1.0   1.0   1.0   1.0
  0.0  -1.0  -1.0   1.0  -1.0  -1.0  …   0.0   0.0   0.0  -1.0   0.0  -1.0
  1.0   0.0  -1.0   1.0  -1.0   0.0      1.0  -1.0   1.0  -1.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0  -1.0     -1.0   1.0  -1.0  -1.0  -1.0   0.0
  0.0  -1.0   0.0   0.0  -1.0  -1.0     -1.0   0.0  -1.0  -1.0  -1.0   1.0
 -1.0  -1.0   1.0   1.0   1.0   0.0     -1.0   1.0   0.0   0.0   0.0  -1.0

*Since the outcomes $j$ are peridoically unobserved, there are two ways to set up the EM problem:*
1. *Define a composite state variable $s=(k,j)$ that is partially unobserved and define $\alpha$ and $\beta$ over this composite state*
2. *Define $\alpha$ and $\beta$ over $k$ only and sum over potential realisations of $j$ when missing*.

*When the number of discrete outcomes is larger, it becomes more difficult to integrate them out when missing. Since $j$ is binary, we will use the second approach.*

#### **Part 1**

*Write a function that (given a guess of the parameters $p$) takes a sequence of $(j^*)_{t=1}^T$ and runs the forward-back algorithm. In doing so, your function should fill in three objects:*
1. *A $K$ x $T$ array of backward looking probabilities $(\alpha)$*
2. *A $K$ x $T$ array of forward looking probabilities $(\beta)$*
3. *A $K$ x $T$ array of posterior probabilities over each state $k(Q)$*.

*Recall that*
$$
\alpha[k,s] = \mathbb{P}[k_s=k,(j^*)_{t=1}^s]
$$
$$
\beta[k,s] = \mathbb{P}[(j^*)_{t=s+1}^T|k_s=k]
$$
and
$$
Q[k,s] = \mathbb{P}[k_s=k|(j^*)_{t=1}^T]
$$


In [2]:
using LinearAlgebra
P_guess = zeros(K,1) 
P_guess .= 0.5
α_init = zeros(K,1)
α_init[1] = 1
β_end = ones(K,1)

############################################################################################################

function fb(P_guess, J_data, n , α_init, β_end)
    T,N = size(J_data)
    K = length(P_guess)
    α, β = zeros(K,T+1), zeros(K,T+1)
    α[:,1] .= α_init
    β[:,T+1] .= β_end
    Q = zeros(K,T)
    #############################
    for j in 1:1
        if j == 1
            if J_data[j,n] == 0
                α[:,2] .= α[:,1]
            elseif J_data[j,n] == 1
                α[2:K,2] .= α[1:K-1,1]
                α[K,2] += α[K,1]
            elseif J_data[j,n] == -1
                α[1,2] = α[1,1] * (1 - P_guess[1])
                α[2,2] = α[1,1] * P_guess[1]
            end
        end
    end
    for j in 2:T
        if J_data[j,n] == 0
            α[:,j+1] .= α[:,j]
        elseif J_data[j,n] == 1
            α[2:K,j+1] = α[1:K-1,j]
            α[K,j+1] += α[K,j] 
        elseif J_data[j,n] == -1
            α_temp_0 = zeros(K,1)
            α_temp_1 = zeros(K,1)
            α_temp_0[1:K-1] = α[1:K-1,j] .* (1 .- P_guess[1:K-1])
            α_temp_1[2:K] = α[1:K-1,j] .* P_guess[1:K-1]
            α_temp_1[K] += α[K,j]
            α[:,j+1] = α_temp_0 .+ α_temp_1
        end
    end
    #############################
    for j in reverse(1:T)
        j_today = J_data[j,n]
        if j_today == -1 
            β[:,j] .= 0.5 .* β[:,j+1]
        elseif j_today == 0 
            β[:,j] .= 0.25 .* β[:,j+1]
        elseif j_today == 1 
            β[:,j] .= 0.25 .* β[:,j+1]
        end
    end
    ###############################
    for t in 1:T
        Q[:,t] = α[:,t] .* β[:,t]
        Q[:,t] = Q[:,t] ./ sum(Q[:,t])
    end
    ###############################
    return α[:, end-9:end], β[:, 1:10], Q
end

############################################################################################################

ex_α, ex_β, ex_Q = fb(P_guess,J_data,747,α_init,β_end)

([0.5 0.0 … 0.0 0.0; 0.5 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.03125 0.0; 0.0 0.0 … 0.96875 1.0], [3.0517578125e-5 6.103515625e-5 … 0.125 0.25; 3.0517578125e-5 6.103515625e-5 … 0.125 0.25; … ; 3.0517578125e-5 6.103515625e-5 … 0.125 0.25; 3.0517578125e-5 6.103515625e-5 … 0.125 0.25], [1.0 0.5 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.0625 0.03125; 0.0 0.0 … 0.9375 0.96875])

#### **Part 2**

*Write a function that iterates over all observations and calculates posterior state probabilities for every observation. i.e. fill in a $K$ x $T$ x $N$ array of posterior weights.*

The function below takes in empty matrices of size $K$ x $T$ x $N$, calculates $\alpha$, $\beta$ and $Q$ for all simulated paths,and fills in the 3d matrices.

In [3]:
α_obs = zeros(K,10,1000)
β_obs = zeros(K,10,1000)
Q_obs = zeros(K,10,1000)

############################################################################################################

function iterate(α_obs, β_obs, Q_obs, P_guess, J_data, α_init, β_end)
    for n in 1:1000
        α_obs[:,:,n], β_obs[:,:,n], Q_obs[:,:,n] = fb(P_guess, J_data, n, α_init, β_end)
    end
    Q_obs = reshape(Q_obs, (1000,10,5))
    return α_obs, β_obs, Q_obs
end

############################################################################################################

α_out, β_out, Q_out = iterate(α_obs, β_obs, Q_obs, P_guess, J_data, α_init, β_end)

([0.5 0.25 … 0.0 0.0; 0.5 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.25 0.15625; 0.0 0.0 … 0.6875 0.8125;;; 0.5 0.25 … 0.0 0.0; 0.5 0.5 … 0.03125 0.015625; … ; 0.0 0.0 … 0.3125 0.234375; 0.0 0.0 … 0.5 0.65625;;; 0.5 0.25 … 0.03125 0.0; 0.5 0.5 … 0.15625 0.03125; … ; 0.0 0.0 … 0.3125 0.3125; 0.0 0.0 … 0.1875 0.5;;; … ;;; 0.5 0.25 … 0.0 0.0; 0.5 0.5 … 0.00390625 0.00390625; … ; 0.0 0.0 … 0.109375 0.109375; 0.0 0.0 … 0.85546875 0.85546875;;; 0.5 0.0 … 0.0 0.0; 0.5 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.0625 0.0625; 0.0 0.0 … 0.9375 0.9375;;; 0.5 0.25 … 0.0 0.0; 0.5 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.09375 0.0546875; 0.0 0.0 … 0.890625 0.9375], [3.0517578125e-5 6.103515625e-5 … 0.125 0.5; 3.0517578125e-5 6.103515625e-5 … 0.125 0.5; … ; 3.0517578125e-5 6.103515625e-5 … 0.125 0.5; 3.0517578125e-5 6.103515625e-5 … 0.125 0.5;;; 6.103515625e-5 0.0001220703125 … 0.25 0.5; 6.103515625e-5 0.0001220703125 … 0.25 0.5; … ; 6.103515625e-5 0.0001220703125 … 0.25 0.5; 6.103515625e-5 0.0001220703125 … 0.25 0.5;;; 3.0517578125e-

*Take as given a set of posterior weights $q_{ntk}=Q[n,t,k]$. The expected log-likelihood for the M-step is:*
$$
\mathcal{L}(p) = \sum_n \sum_t \sum_k q_{ntk}(\mathbf{1}\{j^*_{nt}=1\}\log(p_k)+\mathbf{1}\{j^*_{nt}=0\}\log(1-p_k))
$$
*Show that the non-parametric maximum likelihood estimate of $p$ given the posterior weights is a frequency estimator:*
$$
\hat{p_k} = \frac{\sum_n \sum_t \mathbf{1}\{j^*_{nt}=1\}q_{ntk}}{\sum_n \sum_t \mathbf{1}\{j^*_{nt}\neq-1\}q_{ntk}}
$$
*Write a function to calculate this frequency estimator given posterior weights.*

In [4]:
j_observed_1 = zeros(10,1000)
j_observed_1 .= J_data .== 1
j_observed = zeros(10,1000)
j_observed .= J_data .!= -1

############################################################################################################

function probs(α_obs, β_obs, Q_obs, P_guess, J_data, α_init, β_end)
    #@show size(Q_obs)
    P_out = zeros(5,1)
    Q_new = iterate(α_obs, β_obs, Q_obs, P_guess, J_data, α_init, β_end)[3]
    #@show size(Q_new)
    #@show size(P_guess)
    for k in 1:K
        Q_in = Q_new[:,:,k]
        num = j_observed_1 * Q_in
        den = j_observed * Q_in
        P_out[k] = sum(sum(num, dims = 1), dims = 2)[] ./ sum(sum(den, dims = 1), dims = 2)[]
    end
    return P_out
end

############################################################################################################

out = probs(α_obs, β_obs, Q_obs, P_guess, J_data, α_init, β_end)
    

5×1 Matrix{Float64}:
 0.46683824165885857
 0.46903042606128603
 0.46367854277239245
 0.46850248045068527
 0.4699661534000267

This function updates our guess for the true parameters, $\hat{p}$.

#### **Part 4**

*Run the E-M routine by iterating on this E-step and M-step until you get convergence in $\hat{p}$. Does it look like you can recover the true parameters?*

In [6]:
function EM(J_data, α_init, β_end, toler, max_iter)

    P_init = fill(0.9, K, 1)
    @show P_init
    α_iter = zeros(K,10,1000)
    β_iter = zeros(K,10,1000)
    Q_iter = zeros(K,10,1000)

    error = 1.0
   
    for iter in 1:max_iter

        α_iter, β_iter, Q_iter = iterate(α_iter, β_iter, Q_iter, P_init, J_data, α_init, β_end)
        Q_iter = reshape(Q_iter, (5,10,1000))
        P_update = probs(α_iter, β_iter, Q_iter, P_init, J_data, α_init, β_end)

        error = maximum(abs, P_update - P_init)

        if iter % 10 == 0
            println("Iteration $iter: error = $error")
        end
        
        if error < toler && iter > 10 #### added iter > 10 only to avoid the error = 0.0 instantly which is not true
            println("Converged after $iter iterations")
            return P_init
        end

        iter += 1

        P_init = copy(P_update)

    end

    return P_init
end

P = EM(J_data, α_init, β_end, 1e-6, 1000)

P_init = [0.9; 0.9; 0.9; 0.9; 0.9;;]
Iteration 10: error = 1.1102230246251565e-16
Converged after 11 iterations


5×1 Matrix{Float64}:
 0.4663688497284882
 0.4688451456796985
 0.46296054173478446
 0.4681993319114479
 0.469577972593547

We fail to recover the true parameters, $\hat{p}$.